- Humor Crawler : natepan (10대이야기, 20대이야기, 톡커들의 선택 명예의 전당 (일별)), todayhumor (베오베, 베스트게시물), ilbe (일베-일간베스트), dcinside (야구갤러리), ppomppu (자유게시판)

- 오유 베오베 : 1
- 오유 베게 : 2
- 일베-일간베스트 : 3
- dcinside (야구갤러리) : 4
- 뽐뿌 자유게시판 : 5

In [ ]:
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime

def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params = params if method.upper() == 'GET' else '',
                               data = params if method.upper() == 'POST' else '',
                               headers = headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60) # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [ ]:
conn = sqlite3.connect('Humor.db')
cur = conn.cursor()
# cur.executescript('''
#     DROP TABLE IF EXISTS head;
#     CREATE TABLE head(
#         pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
#         head TEXT NOT NULL,
#         wdate TEXT NOT NULL,
#         cdate TEXT NOT NULL,
#         ref INTEGER NOT NULL,
#         page INTEGER NOT NULL
#     );
    
#     DROP TABLE IF EXISTS history;
#     CREATE TABLE history(
#         pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
#         seen TEXT NOT NULL
#     )
# ''')

In [ ]:
# 오늘의 유머 베스트오브베스트
url = 'http://www.todayhumor.co.kr/board/list.php?table=bestofbest&page=1'

urls = list()
seen = list()

urls.append(url)


count = 0

while urls:
    try:
        count += 1
        seed = urls.pop(0)
        seen.append(seed)
        cur.execute('INSERT INTO history(seen) VALUES(?)', [seed])
        conn.commit()

        resp = download(seed)
        dom = BeautifulSoup(resp.text, 'lxml')


        for _ in [_['href'] for _ in dom.select('tbody > tr:nth-of-type(33) a')
                  if _.has_attr('href') and re.match(r'(^l.+)',_['href']).group()]:
            newUrls = urljoin(seed,_)
            if newUrls not in urls and newUrls not in seen:
                urls.append(newUrls)
                
        if dom.select_one('td.subject > a'):
            head = [_.text.strip() for _ in dom.select('td.subject > a')]
            date = [_.text.strip() for _ in dom.select('tbody > tr > td.date')]
            for _ in range(0,len(date)):
                cur.execute('INSERT INTO head(head, wdate, cdate,page, ref) VALUES(?,?,?,?,1)',
                            [head[_], date[_], str(datetime.datetime.now()).split('.')[0], re.search(r'=(\d+)',urlparse(seed)[4]).group(1)])
                conn.commit()
        
        if count%100 == 0:
            print(count)

    except Exception as e:
        print(e)

In [ ]:
# 오늘의 유머 베스트게시물
url = 'http://www.todayhumor.co.kr/board/list.php?table=humorbest&page=1'

urls = list()
seen = list()

urls.append(url)


count = 0

while urls:
    try:
        count += 1
        seed = urls.pop(0)
        seen.append(seed)
        cur.execute('INSERT INTO history(seen) VALUES(?)', [seed])
        conn.commit()

        resp = download(seed)
        dom = BeautifulSoup(resp.text, 'lxml')


        for _ in [_['href'] for _ in dom.select('tbody > tr:nth-of-type(33) a')
                  if _.has_attr('href') and re.match(r'(^l.+)',_['href']).group()]:
            newUrls = urljoin(seed,_)
            if newUrls not in urls and newUrls not in seen:
                urls.append(newUrls)
                
        if dom.select_one('td.subject > a'):
            head = [_.text.strip() for _ in dom.select('td.subject > a')]
            date = [_.text.strip() for _ in dom.select('tbody > tr > td.date')]
            for _ in range(0,len(date)):
                cur.execute('INSERT INTO head(head, wdate, cdate,page, ref) VALUES(?,?,?,?,2)',
                            [head[_], date[_], str(datetime.datetime.now()).split('.')[0], re.search(r'=(\d+)',urlparse(seed)[4]).group(1)])
                conn.commit()
        
        if count%100 == 0:
            print(count)

    except Exception as e:
        print(e)

In [ ]:
# 일베-일간베스트
url = 'https://www.ilbe.com/list/ilbe'
params = {
    'page':'',
    'listStyle':'list',
}

page = list()
cur.execute('SELECT seen FROM history')
page.append(int(cur.fetchall()[-1][0])+1)

seen = list()


count = page[0]

while page:
    try:
        params['page'] = page.pop(0)
        count += 1
        seen.append(params['page'])
        cur.execute('INSERT INTO history(seen) VALUES(?)', [params['page']])
        conn.commit()

        resp = download(url, params=params, method='GET')
        dom = BeautifulSoup(resp.text, 'lxml')
        
        if '{}'.format(count) in [_.text.strip() for _ in dom.select('.paginate > a') if _.has_attr('class') == False]:
            page.append(count)
        if count % 10 == 1:
            page.append(count)
            
        head = [_.text.strip() for _ in dom.select('.board-body .subject') if _.has_attr('style') == False]
        date = [_.text.strip() for _ in dom.select('li > .date')][7:]
                
        if len(head)==len(date):
            for _ in range(0,len(date)):
                cur.execute('INSERT INTO head(head, wdate, cdate,page, ref) VALUES(?,?,?,?,3)',
                            [head[_], date[_], str(datetime.datetime.now()).split('.')[0], params['page']])
                conn.commit()
            
        if count%100 == 0:
            print(count)

    except Exception as e:
        print(e)

In [ ]:
# 디시인사이드 - 국내야구갤러리
url = 'https://gall.dcinside.com/board/lists/?id=baseball_new9&page=1'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}


urls = list()
seen = list()

urls.append(url)

count = 0

while urls:
    try:
        count += 1
        seed = urls.pop(0)

        resp = download(seed, headers=headers)
        dom = BeautifulSoup(resp.text, 'lxml')
        seen.append(seed)
        cur.execute('''
            INSERT INTO history(seen) VALUES(?)
        ''', [seed])
        
        for _ in [_['href'] for _ in dom.select('div.bottom_paging_box > a')
                  if _.has_attr('href') and re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+',_['href']).group()]:
            newUrls = urljoin(seed, _)
            if newUrls not in urls and newUrls not in seen:
                 urls.append(newUrls)
        
        if dom.select('tr.ub-content.us-post > td:nth-of-type(2) > a:nth-of-type(1)') != None:
            head = [_.text.strip() for _ in dom.select('tr.ub-content.us-post > td:nth-of-type(2) > a:nth-of-type(1)') ]
            wdate = [_['title'] for _ in dom.select('tr.ub-content.us-post > td.gall_date')]
            cdate = str(datetime.datetime.now()).split('.')[0]
            page = re.search('page=(\d+)', urlparse(seed).query).group(1)
            for _ in range(0,len(head)):
                cur.execute('''
                    INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,4)
                    ''', [head[_], wdate[_], cdate, page])
                conn.commit()

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print(e)

In [ ]:
# 뽐뿌 - 자유게시판
url = 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1321'
headers = {'user-agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:78.0) Gecko/20100101 Firefox/78.0'}
urls = list()
seen = list()

urls.append(url)


count = 0

while urls:
    try:
        count += 1
        seed = urls.pop(0)
        seen.append(seed)
        cur.execute('INSERT INTO history(seen) VALUES(?)', [seed])
        conn.commit()

        resp = download(seed, headers=headers)
        dom = BeautifulSoup(resp.text, 'lxml')


        for _ in [_['href'] for _ in dom.select('#page_list a')
              if _.has_attr('href') and re.match(r'(?:https?:/)?/?\w+(?:[./]\w+)+', _['href'])]:
            newUrls = urljoin(seed,_)
            if newUrls not in urls and newUrls not in seen:
                urls.append(newUrls)
        
        head = [_.text.strip() for _ in dom.select('tr[class^=list] a > font') if _.find('img') == None or _.find('img').has_attr('style') == False]
        date = [_['title'] for _ in dom.select('tr[class^=list] td.eng') if _.has_attr('title') and _['title'][:2] != '13']
            
        if len(head) == len(date):
            for _ in range(0,len(date)):
                cur.execute('INSERT INTO head(head, wdate, cdate,page, ref) VALUES(?,?,?,?,5)',
                            [head[_], date[_], str(datetime.datetime.now()).split('.')[0], re.search(r'=(\d+)',urlparse(seed)[4]).group(1)])
                conn.commit()
        
        if count%100 == 0:
            print(count)

    except Exception as e:
        print(e)